In [ ]:
from urllib.request import urlopen
import json
import urllib.parse
import urllib.request
#import os.path
#from os import path
from pathlib import Path
#import os
import time
import socket

In [ ]:
def get_data(symbol):
    t = time.perf_counter()
    main_dir = str(Path.home() / "Downloads")
    os.chdir(main_dir)
    symbol = urllib.parse.quote(symbol.encode('utf8'))
    url = f'https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category=3&Childs=false&CompanyState=0&CompanyType=-1&Consolidatable=true&IsNotAudited=false&Isic=571919&Length=-1&LetterType=-1&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber=1&Publisher=false&Symbol={symbol}&TracingNo=-1&search=true'
    with urlopen(url) as response:
        body = response.read()
    items = json.loads(body)
    page_number = items['Page']
    total = items['Total']
    ticker = items['Letters'][0]['Symbol']
    print(f'total pages: {page_number}, total files: {total}')
    
   
        
    # create ticker directory
    if path.exists(f'{ticker}') == False:
        os.mkdir(f'{ticker}')
    else:
        pass
    
    # change to ticker directory
    try:
        ticker_dir = f'{main_dir}\{ticker}'
        os.chdir(ticker_dir)
    except:
        print('there is no {ticker} directory')
        
    # files in directory
    if path.exists(f'{main_dir}\{ticker}') == True:
        initial_files_count = len(os.listdir(ticker_dir))
    else:
        pass
    # iterate through page
    count = 0
    broken_links = []
    for page in range(page_number):
        url = f'https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category=3&Childs=false&CompanyState=0&CompanyType=-1&Consolidatable=true&IsNotAudited=false&Isic=571919&Length=-1&LetterType=-1&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber={page+1}&Publisher=false&Symbol={symbol}&TracingNo=-1&search=true'
        print(f'page : {page+1}')
        with urlopen(url) as response:
            body = response.read()
        items = json.loads(body)
        # download files
        for i in range(len(items['Letters'])):
            link = items['Letters'][i]['ExcelUrl']
            title = items['Letters'][i]['Title'].replace('/', '_')
            ticker = items['Letters'][i]['Symbol']
            TracingNo = items['Letters'][i]['TracingNo']
            
            socket.setdefaulttimeout(5)
            try:
                if path.exists(f'{ticker}_{TracingNo}_{title}.xls') == False:
                    urllib.request.urlretrieve(link, filename = f'{ticker}_{TracingNo}_{title}.xls')
                    count += 1
                    print(f'{count}: {ticker}_{TracingNo}_{title}')
                else:
                    pass
            except:
                print(f'broken link, skipped {ticker}_{TracingNo}_{title}')
                broken_links.append(f'{ticker}_{TracingNo}_{title}')
                pass

    final_files_count = len(os.listdir(ticker_dir))
    new_file_counts = final_files_count - initial_files_count
    # bakc to main directory
    os.chdir(main_dir)
    print(f'back to project directory: {os.getcwd()}')
    print(f'initial files count: {initial_files_count}')
    print(f'new file(s) downloaded: {new_file_counts}')
    print(f'number of broken links: {len(broken_links)}')
    for broken in broken_links:
        print(f' broken link # {broken_links.index(broken)+1}: {broken}')
    response.close()
    t2 = time.perf_counter() - t
    print(f'total time taken: {t2:0.2f}  seconds')

In [ ]:
get_data('فولاد')

In [ ]:
import asyncio
import aiohttp
import nest_asyncio
import time
from urllib.request import urlopen
import urllib.parse
import urllib.request
import json
from pathlib import Path
import os.path
from os import path
import socket

###############################################################################
nest_asyncio.apply()
start = time.time()

symbol = 'فولاد'

main_dir = str(Path.home() / "Downloads")
os.chdir(main_dir)
symbol = urllib.parse.quote(symbol.encode('utf8'))
url = f'https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category=3&Childs=false&CompanyState=0&CompanyType=-1&Consolidatable=true&IsNotAudited=false&Isic=571919&Length=-1&LetterType=-1&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber=1&Publisher=false&Symbol={symbol}&TracingNo=-1&search=true'
with urlopen(url) as response:
    body = response.read()
items = json.loads(body)
page_number = items['Page']
total = items['Total']
ticker = items['Letters'][0]['Symbol']

print(f'total pages: {page_number}, total files: {total}')

# create ticker directory
if path.exists(f'{ticker}') == False:
    os.mkdir(f'{ticker}')
else:
    pass

# change to ticker directory
try:
    ticker_dir = f'{main_dir}\{ticker}'
    os.chdir(ticker_dir)
except:
    print('there is no {ticker} directory')

# files in directory
if path.exists(f'{main_dir}\{ticker}') == True:
    initial_files_count = len(os.listdir(ticker_dir))
else:
    pass

###############################################################################
urls = []
for page in range(page_number):
    link = f'https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category=3&Childs=false&CompanyState=0&CompanyType=-1&Consolidatable=true&IsNotAudited=false&Isic=571919&Length=-1&LetterType=-1&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber={page+1}&Publisher=false&Symbol={symbol}&TracingNo=-1&search=true'
    urls.append(link)
###############################################################################
async def fetch(session, url):
    async with session.get(url) as response:
        return await response.read()
    
async def main():
    
    tasks = []
    async with aiohttp.ClientSession() as session:
        for url in urls:
            tasks.append(fetch(session, url))
        htmls = await asyncio.gather(*tasks)
        
        count = 0
        excel_links = []
        broken_links = []
        for html in htmls:
            print(f'page: {htmls.index(html)+1}')
            itemss = json.loads(html)
            
            for i in range(len(itemss['Letters'])):
                excel_link = itemss['Letters'][i]['ExcelUrl']
                title = itemss['Letters'][i]['Title'].replace('/', '_')
                ticker = itemss['Letters'][i]['Symbol']
                TracingNo = items['Letters'][i]['TracingNo']
                excel_links.append(excel_link)
                socket.setdefaulttimeout(5)
                try:
                    if path.exists(f'{ticker}_{TracingNo}_{title}.xls') == False:
                        urllib.request.urlretrieve(excel_link, filename = f'{ticker}_{TracingNo}_{title}.xls')
                        count += 1
                        print(f'{count}: {ticker}_{TracingNo}_{title}')
                    else:
                        pass
                except:
                    print(f'broken link, skipped {ticker}_{TracingNo}_{title}')
                    broken_links.append(f'{ticker}_{TracingNo}_{title}')
                    pass

        final_files_count = len(os.listdir(ticker_dir))
        new_file_counts = final_files_count - initial_files_count
        os.chdir(main_dir)
        print(f'back to project directory: {os.getcwd()}')
        print(f'initial files count: {initial_files_count}')
        print(f'new file(s) downloaded: {new_file_counts}')
        print(f'number of broken links: {len(broken_links)}')
        for broken in broken_links:
            print(f' broken link # {broken_links.index(broken)+1}: {broken}')
            
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    t = time.perf_counter()
    loop.run_until_complete(main())
    t2 = time.perf_counter() - t
    print(f'total time taken: {t2:0.2f}  seconds')

In [ ]:
import asyncio
import aiohttp
import nest_asyncio
import time
from urllib.request import urlopen
import urllib.parse
import urllib.request
import json
from pathlib import Path
import os.path
from os import path
import socket

###############################################################################
nest_asyncio.apply()
start = time.time()

##########################
symbols = ['فولاد', 'فملی']
pages = []
for symbol in symbols:
    main_dir = str(Path.home() / "Downloads")
    os.chdir(main_dir)
    symbol_converted = urllib.parse.quote(symbol.encode('utf8'))
    url = f'https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category=3&Childs=false&CompanyState=0&CompanyType=-1&Consolidatable=true&IsNotAudited=false&Isic=571919&Length=-1&LetterType=-1&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber=1&Publisher=false&Symbol={symbol_converted}&TracingNo=-1&search=true'
    with urlopen(url) as response:
        body = response.read()
    items = json.loads(body)
    page_number = items['Page']
    pages.append(page_number)
    total = items['Total']
    ticker = items['Letters'][0]['Symbol']
    print(f'{symbol}, total pages: {page_number}, total files: {total}')
    # create ticker directory
    if path.exists(f'{ticker}') == False:
        os.mkdir(f'{ticker}')
    else:
        pass

    # change to ticker directory
    try:
        ticker_dir = f'{main_dir}\{ticker}'
        os.chdir(ticker_dir)
    except:
        print('there is no {ticker} directory')

    # files in directory
    if path.exists(f'{main_dir}\{ticker}') == True:
        initial_files_count = len(os.listdir(ticker_dir))
    else:
        pass
    ##########################
urls = []    
for n in pages:
    for p, symbol in zip(range(n), symbols):
        link = f'https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category=3&Childs=false&CompanyState=0&CompanyType=-1&Consolidatable=true&IsNotAudited=false&Isic=571919&Length=-1&LetterType=-1&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber={p+1}&Publisher=false&Symbol={symbol}&TracingNo=-1&search=true'
        urls.append(link)
    
        
##########################

###############################################################################
async def fetch(session, url):
    async with session.get(url) as response:
        return await response.read()
    
async def main():
    
    tasks = []
    async with aiohttp.ClientSession() as session:
        for url in urls:
            tasks.append(fetch(session, url))
        htmls = await asyncio.gather(*tasks)
        
        count = 0
        excel_links = []
        broken_links = []
        for html in htmls:
            print(f'page: {htmls.index(html)+1}')
            itemss = json.loads(html)
            
            for i in range(len(itemss['Letters'])):
                excel_link = itemss['Letters'][i]['ExcelUrl']
                title = itemss['Letters'][i]['Title'].replace('/', '_')
                ticker = itemss['Letters'][i]['Symbol']
                TracingNo = items['Letters'][i]['TracingNo']
                excel_links.append(excel_link)
                socket.setdefaulttimeout(5)
                try:
                    if path.exists(f'{ticker}_{TracingNo}_{title}.xls') == False:
                        os.chdir(ticker_dir)
                        urllib.request.urlretrieve(excel_link, filename = f'{ticker}_{TracingNo}_{title}.xls')
                        count += 1
                        print(f'{count}: {ticker}_{TracingNo}_{title}')
                    else:
                        pass
                except:
                    print(f'broken link, skipped {ticker}_{TracingNo}_{title}')
                    broken_links.append(f'{ticker}_{TracingNo}_{title}')
                    pass

        final_files_count = len(os.listdir(ticker_dir))
        new_file_counts = final_files_count - initial_files_count
        os.chdir(main_dir)
        print(f'back to project directory: {os.getcwd()}')
        print(f'initial files count: {initial_files_count}')
        print(f'new file(s) downloaded: {new_file_counts}')
        print(f'number of broken links: {len(broken_links)}')
        for broken in broken_links:
            print(f' broken link # {broken_links.index(broken)+1}: {broken}')
            
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    t = time.perf_counter()
    loop.run_until_complete(main())
    t2 = time.perf_counter() - t
    print(f'total time taken: {t2:0.2f}  seconds')

In [ ]:
symbol = 'فولاد'
symbol = urllib.parse.quote(symbol.encode('utf8'))
url = f'https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category=3&Childs=false&CompanyState=0&CompanyType=-1&Consolidatable=true&IsNotAudited=false&Isic=571919&Length=-1&LetterType=-1&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber=1&Publisher=false&Symbol={symbol}&TracingNo=-1&search=true'
with urlopen(url) as response:
    body = response.read()
items = json.loads(body)
page_number = items['Page']
total = items['Total']
ticker = items['Letters'][0]['Symbol']

print(f'total pages: {page_number}, total files: {total}')

# create ticker directory
if path.exists(f'{ticker}') == False:
    os.mkdir(f'{ticker}')
else:
    pass

# change to ticker directory
try:
    ticker_dir = f'{main_dir}\{ticker}'
    os.chdir(ticker_dir)
except:
    print('there is no {ticker} directory')

# files in directory
if path.exists(f'{main_dir}\{ticker}') == True:
    initial_files_count = len(os.listdir(ticker_dir))
else:
    pass

###############################################################################
urls = []
for page in range(page_number):
    link = f'https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category=3&Childs=false&CompanyState=0&CompanyType=-1&Consolidatable=true&IsNotAudited=false&Isic=571919&Length=-1&LetterType=-1&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber={page+1}&Publisher=false&Symbol={symbol}&TracingNo=-1&search=true'
    urls.append(link)

In [ ]:
for i in urls:
    print(i)

In [ ]:
foo = ['f1', 'f2']
bar = ['b1', 'b2']
for f, b in zip(foo, bar):
    print(f, b)